In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
OUT_PATH: str = '../results/manifolds/raw'

In [4]:
DATA_PATH: str = '../data/imdb'
DATASETS: list = [
    ('train', 'sample.train'),
    ('test', 'sample.test')
]

In [5]:
DIMS: list = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]
MODELS: list = [
    ('base', 'bert-base-uncased'),
    ('textattack', 'textattack/bert-base-uncased-imdb'),
    ('fabriceyhc', 'fabriceyhc/bert-base-uncased-imdb'),
    ('wakaka', 'Wakaka/bert-finetuned-imdb')
]

In [6]:
### Load Datasets into memory

In [7]:
data_config: dict = {
    'polarities': {
      "negative": 0,
      "positive": 1
    },
    'data_label': 'text',
    'target_label': 'sentiment'
}

In [8]:
from typing import Dict
from modules import Data

datasets: Dict[str, Data] = {
    label: Data(data_path=f"{DATA_PATH}.{name}.csv", **data_config)
    for label, name in DATASETS
}

In [9]:
for label, dataset in datasets.items():
    display(dataset.data)
    display(dataset.data['sentiment'].value_counts(normalize=True))

,text,sentiment
0,"And it falls squarely into the category of ""aw...",negative
1,This is one seriously disturbed movie. Even Th...,negative
2,"Basically this is an overlong, unfunny, action...",negative
3,Hey if you have a little over an hour to kill ...,negative
4,Did anyone read the script. This has to be som...,negative
...,...,...
1240,"First of all, Jenna Jameson is the best actres...",negative
1241,"I didnt think it was possible, but i have foun...",negative
1242,"OK, I taped this off TV and missed the very st...",negative
1243,"Okay, okay, maybe not THE greatest. I mean, Th...",positive


negative    0.503614
positive    0.496386
Name: sentiment, dtype: float64

,text,sentiment
0,This...... Movie.... Is..... Horrible!!!!!! Yo...,negative
1,At the same time John Russell was playing ranc...,positive
2,This is the best version of Gypsy that has bee...,positive
3,"It's just stories, some we wish happen to us, ...",positive
4,"This film, without doubt, is the clearest exam...",positive
...,...,...
1235,"""Cooley High"" is one of my favorite movies EVE...",positive
1236,The Comic Strip featured actors from 'The Youn...,negative
1237,I suppose you could say this film has a grain ...,negative
1238,"Having just watched Acacia, I find that I have...",negative


negative    0.520968
positive    0.479032
Name: sentiment, dtype: float64

In [10]:
### Load Encoder into Memory

In [11]:
encoder_config: dict = {
    'layers': [11]
}

In [12]:
from modules import Encoder

encoders: Dict[str, Encoder] = {
    label: Encoder({**{'model': ref}, **encoder_config})
    for label, ref in MODELS
}

In [13]:
### Compute manifolds and measure centroid distance and cluster dispersion

In [14]:
import numpy as np
import pandas as pd

In [15]:
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [34]:
from scipy.spatial import distance

def metric_computation(record: dict, groups: pd.core.groupby.GroupBy) -> None:

    # compute centroid means and dispersion for each cluster
    for label, group in groups:
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.mean(distance.cdist([record[f'centroid_{label}']], group['reduced_embeds'].tolist()), axis=1).item()

    record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item()

In [17]:
from typing import Generator
import torch

def reduce_analyse(data: pd.DataFrame, col: str, dim: list, default_dim: int = 768) -> Generator:

    embed_col: np.ndarray = torch.stack(data[col].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == default_dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        metric_computation(record, data.groupby(dataset.target_label))

        yield record

In [18]:
output_cols: list = ['dim', 'dispersion_positive', 'dispersion_negative', 'distance']

In [19]:
results: Dict[str, pd.DataFrame] = {}

In [20]:
for enc_label, encoder in encoders.items():
    for data_label, dataset in datasets.items():
        encoder.df_encode(dataset.data, col=dataset.data_label)
        results[f'{data_label}.{enc_label}'] = pd.DataFrame.from_records(
            list(reduce_analyse(
                dataset.data, encoder.col_name, DIMS,
                default_dim=encoder.dim)
            )
        )

In [21]:
for label, dataset in results.items():
    results[label][output_cols].to_csv(f'{OUT_PATH}/{label}.csv')
    display(results[label][output_cols])

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.643370e+06,1.591676e+06,1.259246
1,576,1.644534e+06,1.592719e+06,1.194103
2,384,1.644351e+06,1.592551e+06,1.199964
3,192,1.643708e+06,1.591911e+06,1.219327
4,96,1.642579e+06,1.590627e+06,1.246506
5,48,1.640233e+06,1.588034e+06,1.293415
6,24,1.634228e+06,1.581408e+06,1.443938
7,12,1.622289e+06,1.568095e+06,1.648988
8,6,1.599161e+06,1.537979e+06,1.852708
9,3,1.561676e+06,1.476522e+06,1.520903


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.507873e+06,1.702410e+06,1.229414
1,576,1.509076e+06,1.703589e+06,1.156741
2,384,1.508934e+06,1.703403e+06,1.158986
3,192,1.508509e+06,1.702874e+06,1.169901
4,96,1.507134e+06,1.701512e+06,1.213615
5,48,1.504606e+06,1.698812e+06,1.272759
6,24,1.499124e+06,1.692158e+06,1.409630
7,12,1.489605e+06,1.680789e+06,1.508193
8,6,1.482086e+06,1.662280e+06,0.950279
9,3,1.434469e+06,1.588580e+06,1.229867


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.915401e+06,1.867103e+06,7.820493
1,576,1.915778e+06,1.867371e+06,7.822215
2,384,1.915490e+06,1.867085e+06,7.824227
3,192,1.914635e+06,1.866176e+06,7.829552
4,96,1.912659e+06,1.864185e+06,7.843067
5,48,1.908444e+06,1.859759e+06,7.871172
6,24,1.899243e+06,1.850124e+06,7.931141
7,12,1.878269e+06,1.828336e+06,8.063804
8,6,1.824655e+06,1.774034e+06,8.383164
9,3,1.672867e+06,1.627430e+06,9.144963


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.786823e+06,2.042893e+06,7.312816
1,576,1.787146e+06,2.043185e+06,7.314414
2,384,1.786892e+06,2.042884e+06,7.316451
3,192,1.786042e+06,2.041993e+06,7.322656
4,96,1.784244e+06,2.040015e+06,7.335378
5,48,1.780371e+06,2.035766e+06,7.362444
6,24,1.771795e+06,2.026320e+06,7.423075
7,12,1.752307e+06,2.004841e+06,7.558836
8,6,1.702880e+06,1.950692e+06,7.878849
9,3,1.560394e+06,1.799460e+06,8.708559


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.628460e+06,2.056547e+06,22.864886
1,576,1.628916e+06,2.057076e+06,22.865054
2,384,1.628829e+06,2.057008e+06,22.865054
3,192,1.628542e+06,2.056760e+06,22.865390
4,96,1.627981e+06,2.056267e+06,22.865806
5,48,1.626742e+06,2.055172e+06,22.866762
6,24,1.623751e+06,2.052485e+06,22.869183
7,12,1.616275e+06,2.045783e+06,22.876083
8,6,1.594092e+06,2.024674e+06,22.898670
9,3,1.492467e+06,1.945725e+06,23.058022


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.833460e+06,2.608884e+06,19.964325
1,576,1.833902e+06,2.609408e+06,19.964686
2,384,1.833828e+06,2.609366e+06,19.964686
3,192,1.833563e+06,2.609061e+06,19.965168
4,96,1.832996e+06,2.608601e+06,19.965624
5,48,1.831810e+06,2.607396e+06,19.966653
6,24,1.828797e+06,2.604430e+06,19.970807
7,12,1.821511e+06,2.597674e+06,19.978293
8,6,1.799080e+06,2.574212e+06,20.011916
9,3,1.706871e+06,2.483648e+06,20.243306


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.296890e+06,2.062966e+06,11.483087
1,576,2.297310e+06,2.063295e+06,11.485725
2,384,2.297080e+06,2.063062e+06,11.486758
3,192,2.296474e+06,2.062360e+06,11.488441
4,96,2.294904e+06,2.060776e+06,11.494536
5,48,2.291651e+06,2.057314e+06,11.506000
6,24,2.284118e+06,2.049246e+06,11.537372
7,12,2.266485e+06,2.030528e+06,11.608496
8,6,2.218734e+06,1.980252e+06,11.813304
9,3,2.088940e+06,1.840697e+06,12.382923


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.091581e+06,2.231614e+06,11.398275
1,576,2.092026e+06,2.231988e+06,11.401153
2,384,2.091831e+06,2.231784e+06,11.401629
3,192,2.091119e+06,2.230950e+06,11.405117
4,96,2.089732e+06,2.229337e+06,11.409593
5,48,2.086472e+06,2.225610e+06,11.423860
6,24,2.079016e+06,2.217130e+06,11.455494
7,12,2.060904e+06,2.196837e+06,11.539209
8,6,2.013277e+06,2.144489e+06,11.756336
9,3,1.887557e+06,1.995884e+06,12.355372


In [37]:
import numpy as np

data: np.ndarray = np.random.rand(16, 3)
data

array([[0.83253054, 0.76232724, 0.9311698 ],
       [0.45823262, 0.28139747, 0.73184336],
       [0.8977754 , 0.5910931 , 0.76227186],
       [0.25591634, 0.67925163, 0.13462081],
       [0.61021212, 0.41356852, 0.58002253],
       [0.30884924, 0.33575787, 0.44247615],
       [0.71400359, 0.29860992, 0.61326371],
       [0.39913074, 0.86888994, 0.49518741],
       [0.11551349, 0.49203727, 0.73162897],
       [0.01530828, 0.67913729, 0.89200209],
       [0.26389048, 0.59565207, 0.8133027 ],
       [0.02498959, 0.18602134, 0.58513122],
       [0.39495056, 0.40787673, 0.75417219],
       [0.74055014, 0.74445187, 0.69463685],
       [0.71759715, 0.15285993, 0.58924924],
       [0.7398997 , 0.95635215, 0.47090121]])